## From functional dependencies to candidate keys

In [41]:
fd = "AB->CDE"

R = "A,B,C,D,E,G,H"
# Order righthand letters alphabetically
F = "AB->C,C->D,D->B,A->E,EH->G"


In [42]:
def to_rules(F):
    rules = dict()
    rules_str = F.split(",")
    for rule_str in rules_str:
        parts = rule_str.split("->")
        key= frozenset(parts[0])
        if rules.has_key(key):
            rules[key].update(set(parts[1]))
        else:
            rules[key]=set(parts[1])
    return rules

In [43]:
to_rules(F)

{frozenset({'E', 'H'}): {'G'},
 frozenset({'D'}): {'B'},
 frozenset({'C'}): {'D'},
 frozenset({'A'}): {'E'},
 frozenset({'A', 'B'}): {'C'}}

In [44]:
def preprocessing(fields,rules):
    L=set()
    M=set()
    R=set()
    for field in fields:
        in_left = False
        in_right = False
        for key in rules:
            if field in key:
                in_left=True
            
            if field in rules[key]:
                in_right=True
        
        if in_left and in_right:
            M.update(field)
        elif in_left:
            L.update(field)
        elif in_right:
            R.update(field)
        else: # Not in any rule. Must be in key
            L.update(field)
    return (L,M,R)

preprocessing(set(R.split(",")),to_rules(F))
        

({'A', 'H'}, {'B', 'C', 'D', 'E'}, {'G'})

In [45]:
def can_prune(candidate, candidate_keys):
    for key in candidate_keys:
        if key.issubset(candidate):
            return True
    return False

''' Check if a given candidate is a key '''
def is_key(candidate, fields, rules):
    dependents = set()
    dependents.update(candidate)
    new_dependents=True
    while len(fields.difference(dependents))>0:
        new_dependents=False
        for key in rules.keys():
            if key.issubset(dependents): # Rule can be applied
                if len(rules[key].difference(dependents))>0:
                    new_dependents=True
                    dependents.update(rules[key])
        if new_dependents is False:
            return False
    return True
        

def get_candidate_keys(fields, rules):
    (in_key,maybe_in_key,not_in_key) = preprocessing(fields,rules)
    
    candidate_keys = set()
    openlist = list()
    openlist.append(frozenset(in_key))


    while len(openlist)>0:
        candidate = openlist.pop(0) # Order is important to maintain a breadth-first order on cardinality

        # Can prune?
        if can_prune(candidate, candidate_keys):
            continue
               
        # Is key?
        if is_key(candidate,fields, rules):            
            candidate_keys.add(candidate)
        else:
            # Add children
            for field in maybe_in_key.difference(candidate):
                new_candidate = set()
                new_candidate.add(field)
                new_candidate.update(candidate)
                if can_prune(new_candidate, candidate_keys)==False:
                    openlist.append(frozenset(new_candidate))
    return candidate_keys

In [46]:
fields = set(R.split(","))
rules = to_rules(F)
get_candidate_keys(fields, rules)

{frozenset({'A', 'C', 'H'}),
 frozenset({'A', 'D', 'H'}),
 frozenset({'A', 'B', 'H'})}